# Neural Networks and 2x2 matrices

In this session we will try to approximate a 2-by-2 matrix with a small neural network using $\varphi=\text{ReLU}(x)=\text{max}\lbrace x,0\rbrace$ (Rectified linear units, applied component-wise) as nonlinear activation function.

A small 16-parameter neural net with ReLU-nonlinearity in the first layer, no nonlinearity in the second one can represent a 2-by-2 matrix exactly.



In this exercise we want to employ this net to approximate $A_\varepsilon$ (direct operator) and $A_\varepsilon^{-1}$ (inverse operator) for 

$$A_\varepsilon = \begin{bmatrix}
    1  &  1 \\
    1  &  1 + \varepsilon \\
\end{bmatrix}
$$

and $\varepsilon = 10^0, 10^{-1}, 10^{-2}, 10^{-3},...$

### Exercise
**a)** For fixed $\epsilon > 0$ and $\sigma > 0$ generate a data set $(x_i, y_i)$ with $x_i \sim \mathcal{N}(0, 1)$ and $y_i^\delta = A_\varepsilon x_i + \sigma \eta_i$, $\eta_i \sim \mathcal{N}(0,1)$ for $i=1,...,10^3$. Create two plots, one containing the vectors $x_i$ and the other the vectors $y_i$.

In [ ]:
# !!Your code goes here







**b)** For each value of $\varepsilon$ generate the data set and split it as in the previous exercises into training and validation set. Then train the network with the previously described architecture to approximate $A_\varepsilon$. Create one plot with all the validation error curves corresponding to the different values of $\varepsilon$.

In [ ]:
# !!Your code goes here







**c)** Do the same for $A_\varepsilon^{-1}$.

In [ ]:
# !!Your code goes here








